In [ ]:
import logging
import os
import shutil
import traceback
import tempfile
from db_models import File, FileLocation, FilingTag, FileTagLabel, FileEmbedding, get_db_engine
from embedding.minilm import MiniLMEmbedder
from sqlalchemy import func
from sqlalchemy.orm import Session
from text_extraction.pdf_extraction import PDFTextExtractor
from text_extraction.basic_extraction import TextFileTextExtractor, TikaTextExtractor, get_extractor_for_file
from text_extraction.image_extraction import ImageTextExtractor
from text_extraction.office_doc_extraction import PresentationTextExtractor, SpreadsheetTextExtractor, WordFileTextExtractor
from text_extraction.web_extraction import HtmlTextExtractor, EmailTextExtractor
from text_extraction.extraction_utils import common_char_replacements, strip_diacritics, normalize_unicode, normalize_whitespace
from typing import Optional
from logging_setups import setup_logger

nb_logger = setup_logger(name="NotebookLogger", notebook=True, level=logging.DEBUG)

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

pdf_extractor = PDFTextExtractor()
txt_extractor = TextFileTextExtractor()
image_extractor = ImageTextExtractor()
presentation_extractor = PresentationTextExtractor()
spreadsheet_extractor = SpreadsheetTextExtractor()
word_extractor = WordFileTextExtractor()
html_extractor = HtmlTextExtractor()
email_extractor = EmailTextExtractor()
tika_extractor = TikaTextExtractor()

extractors_list = [
    pdf_extractor,
    txt_extractor,
    image_extractor,
    presentation_extractor,
    spreadsheet_extractor,
    word_extractor,
    html_extractor,
    email_extractor
]

def get_files_from_tag_locations(
        db_session: Session,
        tag_obj: FilingTag,
        n: int = 100,
        randomize: bool = False,
        exclude_embedded: bool = False,
        max_size_mb: Optional[float] = None
        ):
    """
    Retrieve files tagged with a specific tag string.
    Optionally exclude files that already have a FileEmbedding.
    """
    tag_locations = FileLocation.file_server_directories.ilike(f"%/{tag_obj.full_tag_label_str}%")

    # start the base query
    q = db_session.query(File) \
                  .join(FileLocation) \
                  .filter(tag_locations)

    if exclude_embedded:
        # left‐outer join to file_embeddings, keep only those with no match
        q = q.outerjoin(FileEmbedding, File.hash == FileEmbedding.file_hash)\
            .filter(FileEmbedding.file_hash == None)\
            
    if max_size_mb is not None:
        max_size_bytes = max_size_mb * 1024 * 1024
        q = q.filter(File.size <= max_size_bytes)

    if randomize:
        q = q.order_by(func.random())
    
    q = q.limit(n)

    return q.all()

def label_file_using_tag(
        db_session: Session,
        file_obj: File,
        some_tag: FilingTag|str,
        label_source: str = 'rule'
    ):
    """
    Label a file with a specific tag.
    """
    
    if isinstance(some_tag, str):
        tag_obj = FilingTag.retrieve_tag_by_label(db_session, some_tag)
        if not tag_obj:
            raise ValueError(f"Tag '{some_tag}' not found in the database.")
        current_tag = tag_obj
    elif isinstance(some_tag, FilingTag):
        tag_obj = some_tag
        current_tag = tag_obj
    else:
        raise TypeError("some_tag must be a FilingTag instance or a string representing the tag label.")
    
    while current_tag is not None:
        already = (
            db_session.query(FileTagLabel)
            .filter_by(file_id=file_obj.id, tag=current_tag.label)
            .first()
        )
        if already:
            nb_logger.info(f"File {file_obj.id} already labeled with tag {current_tag.label}")
            current_tag = current_tag.parent
            continue

        labeling_record = FileTagLabel(
            file_id=file_obj.id,
            file_hash=file_obj.hash,
            tag=current_tag.label,
            is_primary=True if not current_tag.parent else False,
            label_source=label_source
        )
        db_session.add(labeling_record)
        nb_logger.info(f"Added tag {current_tag.label} to file with hash {file_obj.hash}")
        current_tag = current_tag.parent

    db_session.commit()
    return labeling_record


filing_code_tag = "D4 - Mitigation Monitoring Program"
text_leng_threshold = 250
file_server_location = r"N:\PPDO\Records"
n = 250
embedding_client = MiniLMEmbedder()
with Session(get_db_engine()) as db_session:
    tag = FilingTag.retrieve_tag_by_label(db_session, filing_code_tag)
    files = get_files_from_tag_locations(
        db_session= db_session,
        tag_obj = tag,
        randomize=True,
        n = n,
        exclude_embedded=True,
        max_size_mb=150
    )

    with tempfile.TemporaryDirectory() as temp_dir:
        for file_i, file_obj in enumerate(files):
            nb_logger.info(f"Processing file number {file_i + 1} of {len(files)}: {file_obj.id}")

            if not file_obj.locations:
                nb_logger.warning(f"No locations found for file {file_obj.id}")
                continue
            
            local_location_filepath = ""
            filename = ""
            for location in file_obj.locations:
                # if the tag's full label is not in the file server directories, continue
                if tag.full_tag_label_str.lower() not in location.file_server_directories.lower():
                    continue
                
                local_location_filepath = location.local_filepath(server_mount_path=file_server_location)

                if os.path.exists(local_location_filepath):
                    nb_logger.info(f"Found file {file_obj.id} at {local_location_filepath}")
                    filename = location.filename
                    break
                
                else:
                    nb_logger.warning(f"File {file_obj.id} not found at {local_location_filepath}")
                    continue
            
            # if no valid file found, skip to next file
            if not filename:
                nb_logger.warning(f"No valid file found for {file_obj.id}")
                continue

            try:
                temp_filepath = os.path.join(temp_dir, filename)
                shutil.copyfile(local_location_filepath, temp_filepath)
                extractor = get_extractor_for_file(temp_filepath, extractors_list)
                if extractor:
                    text = extractor(temp_filepath)
                else:
                    text = tika_extractor(temp_filepath)  # Fallback to Tika extractor
                
                # if the text is empty or too short, skip processing
                if text and len(text) >= text_leng_threshold:
                    text = common_char_replacements(text)
                    text = strip_diacritics(text)
                    text = normalize_unicode(text)
                    text = normalize_whitespace(text)
                    emb_list = embedding_client.encode([text])
                    emb_vec = emb_list[0] if emb_list else None
                    if emb_vec is not None:
                        file_embedding = FileEmbedding(
                            file_hash =file_obj.hash,
                            source_text =text,
                            minilm_model=embedding_client.model_name,
                            minilm_emb=emb_vec
                        )
                        db_session.add(file_embedding)
                        db_session.commit()
                        nb_logger.info(f"Processed file {file_obj.id} with tag {filing_code_tag}")
                    else:
                        nb_logger.warning(f"Failed to create embedding for file {file_obj.id}")
                        continue
                    
                    file_labeling = label_file_using_tag(
                        db_session=db_session,
                        file_obj=file_obj,
                        some_tag=tag
                    )
                else:
                    if len(text) < text_leng_threshold:
                        nb_logger.warning(f"Text length for file {file_obj.id} is {len(text)}:\n{text}")
                    else:
                        nb_logger.warning(f"No text extracted from {file_obj.id}")
                    continue
            
            except Exception as e:
                nb_logger.error(f"Error processing file {file_obj.id}: {e}")
                nb_logger.debug(traceback.format_exc())
                continue

INFO - Processing file number 1 of 202: 214868
INFO - Found file 214868 at N:\PPDO\Records\21xx   Central Heating Plant (Fackler Cogeneration)\2113\2113\D - Environmental Review Process\D4 - Mitigation Monitoring Program\UCSC Cogen THP  LTO Amend 8.17.12.pdf
INFO - Processed file 214868 with tag D4 - Mitigation Monitoring Program
INFO - Added tag D4 to file with hash bedc35d8de0eb40b1ce73b64dfe4488809d5cddf
INFO - Added tag D to file with hash bedc35d8de0eb40b1ce73b64dfe4488809d5cddf
INFO - Processing file number 2 of 202: 490397
INFO - Found file 490397 at N:\PPDO\Records\55xx   College 9 & 10\5504\D - Environmental Review Process\D4 - Mitigation Monitoring Program\File 5504.D4 C9-10 Projects Status Update 1-24-94.pdf


Output()

Output()

Output()

Output()

...

Output()

Output()

[tesseract] Error during processing.

[tesseract] Error during processing.

KeyboardInterrupt: 